In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import cross_val_score

In [2]:
df = pd.read_csv('./../data/ames_housing_price_data_v3.csv', index_col = 0)

In [3]:
price = df['SalePrice']
price_log = df['SalePrice_log']

In [ ]:
df2 = df.drop(['SalePrice', 'SalePrice_log', 'sold_datetime'], axis = 1)